# Finding Similar Items

## 1.	Introduction

### Frequent pairs of items

- Extract a list of the authors or editors per publication from the ACL Anthology dataset (https://aclanthology.org/) and create baskets and perform a search on the following:

1. Find the frequent pair of items (2-tuples) using the naïve, A-priori and PCY algorithms. For each of these compare the time of execution and results for supports s=10, 50, 100. Comment your results.

2. For the PCY algorithm, create up to 5 compact hash tables. What is  the difference in results and time of execution for 1,2,3,4 and 5 tables? Comment your results.

3. Find the final list of k-frequent items (k-tuples) for k=3 and 4. Experiment a bit and describe the best value for the support in each case. *Warning*: You can use any of the three algorithms, but be careful, because the algorithm can take too long if you don't chose it properly (well, basically don't use the naïve approach ;)).

4. Using one of the results of the previous items, for one k (k=2 or 3) find the possible clusters using the 1-NN criteria. Comment your results.

> 1-NN means that if you have a tuple {A,B,C} and {C,E,F} then because they share one element {C}, then they belong to the same cluster  {A,B,C,E,F}.

-	Define all and only used package imports below

In [1]:
import requests # to download the dataset
import gzip
import shutil # to extract the gz file
import re # for text cleaning

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random 

import time
import itertools

## 2.	ELT

### Extract, Load and Transform of data.

- In your code data should be retrieved from an online source, NOT from your local drive, otherwise, nobody can run your code without additional effort.

In [2]:
# Download data 
url = 'https://aclanthology.org/anthology+abstracts.bib.gz'
filename = url.split("/")[-1]
with open(filename, "wb") as f:
    r = requests.get(url)
    f.write(r.content)

# Extract the gz file
with gzip.open('anthology+abstracts.bib.gz', 'rb') as f_in:
    with open('anthology+abstracts.bib', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

In [3]:
# Find all the rows in the file that contain an author/editor and load the text to a list
end = set(['",\n', '}\n'])
authors = []
with open("anthology+abstracts.bib", "r",encoding="UTF-8") as f:
    line = f.readline()
    while(line != ''):

        if line.__contains__('author =') or line.__contains__('editor ='):
            while not (line.endswith('",\n')|line.endswith('},\n')):
                line = line+f.readline()
            # something to clean
#           line = LatexNodes2Text().latex_to_text(line) # fix umlaut vocals; this part takes some time to run    
            line = re.sub('    editor = "|    author = "|",|\n|    author = {|    editor = {','',line)
            line = re.sub(',','',line)
            line = re.sub('  and      ',', ',line)
            authors.append(line)
            
        line = f.readline()
        
    f.close()

print("{} baskets of authors were found in the file.".format(len(authors)))

73199 baskets of authors were found in the file.


#In the above task, we have done the data mining & cleaning activity. And we were able to consolidate only authors & editors into one text file

In [4]:
authors[1]

'Singh Sumer, Li Sheng'

#Wtrite function to create frozen sets

In [5]:
def readdata(k, fname=authors, report=False):
    C_k = []
    b = 0

    for line in fname:
        if report:
            print(line)
         
        # gather all items in one basket
        C_k.append(line.split(", "))

        # end of basket, report all itemsets
        for author in C_k:
            for itemset in itertools.combinations(author, k):
                yield frozenset(itemset)
            C_k = []
                
        if report:
            print("")

    # last basket
    if len(C_k) > 0:
        for itemset in itertools.combinations(C_k, k):
            yield frozenset(itemset)
    

### Report the essential description of data.
-	Don’t print out dozens of raw lines.

In [6]:
nitems = 5
for C_k in readdata(k=2, report=True):
    print(C_k)
    nitems -= 1
    
    if nitems == 0:
        break

Mostafazadeh Davani Aida, Kiela Douwe, Lambert Mathias, Vidgen Bertie, Prabhakaran Vinodkumar, Waseem Zeerak
frozenset({'Kiela Douwe', 'Mostafazadeh Davani Aida'})
frozenset({'Lambert Mathias', 'Mostafazadeh Davani Aida'})
frozenset({'Vidgen Bertie', 'Mostafazadeh Davani Aida'})
frozenset({'Prabhakaran Vinodkumar', 'Mostafazadeh Davani Aida'})
frozenset({'Waseem Zeerak', 'Mostafazadeh Davani Aida'})


In [7]:
# pair of elements
import time


def get_C(k):

    start = time.time()
    C = {}
    for key in readdata(k):  # False report
        if key not in C:
            C[key] = 1
        else:
            C[key] += 1
    
    return C


C1 = get_C(1)
C2 = get_C(2)
C3 = get_C(3)
C4 = get_C(4)

In [8]:
print(len(C1),len(C2),len(C3),len(C4))

64850 271422 784913 4376253


In [9]:
for (ck, n), _ in zip(C2.items(), range(5)):
    print(ck,n)

frozenset({'Kiela Douwe', 'Mostafazadeh Davani Aida'}) 2
frozenset({'Lambert Mathias', 'Mostafazadeh Davani Aida'}) 1
frozenset({'Vidgen Bertie', 'Mostafazadeh Davani Aida'}) 2
frozenset({'Prabhakaran Vinodkumar', 'Mostafazadeh Davani Aida'}) 3
frozenset({'Waseem Zeerak', 'Mostafazadeh Davani Aida'}) 2


In [10]:
for (ck, n), _ in zip(C3.items(), range(5)):
    print(ck,n)

frozenset({'Lambert Mathias', 'Kiela Douwe', 'Mostafazadeh Davani Aida'}) 1
frozenset({'Vidgen Bertie', 'Kiela Douwe', 'Mostafazadeh Davani Aida'}) 2
frozenset({'Prabhakaran Vinodkumar', 'Kiela Douwe', 'Mostafazadeh Davani Aida'}) 2
frozenset({'Waseem Zeerak', 'Kiela Douwe', 'Mostafazadeh Davani Aida'}) 2
frozenset({'Vidgen Bertie', 'Lambert Mathias', 'Mostafazadeh Davani Aida'}) 1


## 3.	Modeling

### Prepare analytics here and construct all the data objects you will use in your report.
•	Write functions and classes to simplify tasks. Do not repeat yourself.

•	Avoid output.

•	Refactor your code until it’s clean

In [11]:
def interesting_pairs(s):
    L2 = {}
    C2 = get_C(2)
    for key, n in C2.items():
        if n >= s:
            L2[key] = n
    L2 = [elem for elem in list(L2) if len(elem) > 1] 

    for i in range(len(L2)):

        A, B = list(L2[i])
        support_AB = C2[frozenset([A, B])]
        support_A = C1[frozenset([A])]
        conf_A_leads_to_B = support_AB / support_A

        support_B = C1[frozenset([B])]
        prob_B = support_B / nbaskets

        interest_A_leads_to_B = conf_A_leads_to_B - prob_B

        if interest_A_leads_to_B > 0.7:
            print("{} --> {} with interest {:3f}".format(A, B,
                                                         interest_A_leads_to_B))

•	Naive, Apriori and PCY for 2-tuple frequent item sets (k=2)

In [12]:
def Naive_method(s):
    C2 = get_C(2)
    t = time.time()
    L2_naive = {}
    for key, n in C2.items():
        if n >= s:
            L2_naive[key] = n
    t2 = round(time.time() - t,3)
    print('Naive:    {} items with >{} occurances. Calculated in {} sec'.format(len(L2_naive), s, t2))

def Apriori(s):
    t = time.time()
    # find frequent 1-tuples (individual items)
    C1 = {}
    for key in readdata(k=1, report=False):
        if key not in C1:
            C1[key] = 1
        else:
            C1[key] += 1    
        
    # filter stage
    L1 = {}
    for key, count in C1.items():
        if count >= s:
            L1[key] = count
    C2_items = set([a.union(b) for a in L1.keys() for b in L1.keys()]) # List comprehensions in python
    
    # find frequent 2-tuples
    C2 = {}
    for key in readdata(k=2):
        # filter out non-frequent tuples
        if key not in C2_items:
            continue

        # record frequent tuples
        if key not in C2:
            C2[key] = 1
        else:
            C2[key] += 1

    # filter stage
    L2 = {}
    for key, count in C2.items():
        if count >= s:
            L2[key] = count
    t2 = round(time.time() - t,3)
    print('A-priori: {} items with >{} occurances. Calculated in {} sec'.format(len(L2), s, t2))
    
def PCY(s):
    t = time.time()
    # find frequent 1-tuples (individual items)
    C1 = {}
    for key in readdata(k=1, report=False):
        if key not in C1:
            C1[key] = 1
        else:
            C1[key] += 1    
            
    L1 = {}
    for key, count in C1.items():
        if count >= s:
            L1[key] = count

    C2_items = set([a.union(b) for a in L1.keys() for b in L1.keys()]) # List comprehensions in python

    # hash table
    max_hash1 = 10 * 1000000
    H1 = np.zeros((max_hash1, ), dtype=int)

    for key in readdata(k=2, report=False):
        hash_cell_1 = hash(key) % max_hash1
        H1[hash_cell_1] += 1
    # find frequent 2-tuples
    C2 = {}

    for key in readdata(k=2):
        # hash-based filtering stage from PCY
        hash_cell_1 = hash(key) % max_hash1
        if H1[hash_cell_1] < s:
            continue
    
        # filter out non-frequent tuples
        if key not in C2_items:
            continue

        # record frequent tuples
        if key not in C2:
            C2[key] = 1
        else:
            C2[key] += 1

    # filter stage
    L2 = {}
    for key, count in C2.items():
        if count >= s:
            L2[key] = count
    t2 = round(time.time() - t,3)
    print('PCY:      {} items with >{} occurances. Calculated in {} sec'.format(len(L2), s, t2))

## 4.	Results

•	Print out relevant tables nicely, display well-annotated charts and explain if needed in plain English.
•	Use minimum code here, just output-functions’ calls.

1. Find the frequent pair of items (2-tuples) using the naïve, A-priori and PCY algorithms. For each of these compare the time of execution and results for supports s=10, 50, 100. Comment your results.

In [ ]:
for s in range(10,110,10):
    Naive_method(s)
    Apriori(s)
    PCY(s)


Naive:    1784 items with >10 occurances. Calculated in 0.05 sec


In [ ]:
nbaskets = len(authors)
interesting_pairs(50)

2. For the PCY algorithm, create up to 5 compact hash tables. What is  the difference in results and time of execution for 1,2,3,4 and 5 tables? Comment your results.

In [ ]:
N=20
s = N
t = time.time()
# hash table
max_hash1 = 5*1000000-673
max_hash2 = 5*1000000+673
max_hash3 = 5*1000000+2*673
max_hash4 = 5*1000000+3*673
max_hash5 = 5*1000000+4*673


H1 = np.zeros((max_hash1,), dtype=int)
H2 = np.zeros((max_hash2,), dtype=int)
H3 = np.zeros((max_hash3,), dtype=int)
H4 = np.zeros((max_hash4,), dtype=int)
H5 = np.zeros((max_hash5,), dtype=int)

for key in readdata(k=2, report=False):
    hash_cell_1 = hash(key) % max_hash1
    H1[hash_cell_1] += 1
    hash_cell_2 = hash(key) % max_hash2
    H2[hash_cell_2] += 1
    hash_cell_3 = hash(key) % max_hash3
    H3[hash_cell_3] += 1
    hash_cell_4 = hash(key) % max_hash4
    H4[hash_cell_4] += 1
    hash_cell_5 = hash(key) % max_hash5
    H5[hash_cell_5] += 1

# compact hash table
H_good_1 = set(np.where(H1 >= N)[0])
H_good_2 = set(np.where(H2 >= N)[0])
H_good_3 = set(np.where(H3 >= N)[0])
H_good_4 = set(np.where(H4 >= N)[0])
H_good_5 = set(np.where(H5 >= N)[0])
del H1
del H2
del H3
del H4
del H5

# find frequent 1-tuples (individual items)
C1 = {}
for key in readdata(k=1, report=False):
    if key not in C1:
        C1[key] = 1
    else:
        C1[key] += 1    

# filter stage
L1 = {}
for key, count in C1.items():
    if count >= N:
        L1[key] = count

C2_items = set([a.union(b) for a in L1.keys() for b in L1.keys()]) # List comprehensions in python

# find frequent 2-tuples
C2 = {}


for i in range (1,6):
    for key in readdata(k=2):
        # hash-based filtering stage from PCY
        if i >= 1:
            hash_cell_1 = hash(key) % max_hash1
            if hash_cell_1 not in H_good_1:
                continue
        if i >= 2:
            hash_cell_2 = hash(key) % max_hash2
            if hash_cell_2 not in H_good_2:
                continue
        if i >= 3:
            hash_cell_3 = hash(key) % max_hash3
            if hash_cell_3 not in H_good_3:
                continue
        if i >= 4:
            hash_cell_4 = hash(key) % max_hash4
            if hash_cell_4 not in H_good_4:
                continue
        if i >= 5:
            hash_cell_5 = hash(key) % max_hash5
            if hash_cell_5 not in H_good_5:
                continue
        
     # filter out non-frequent tuples
        if key not in C2_items:
            continue

    # record frequent tuples
        if key not in C2:
            C2[key] = 1
        else:
            C2[key] += 1
        
#print("{} items".format(len(C2)))

# filter stage
    L2 = {}
    for key, count in C2.items():
        if count >= N:
            L2[key] = count
    t2 = round(time.time() - t,3)
    print('{} items with >{} occurances. Number of hash-tables: {}. Calculated in {} sec'.format(len(L2), s, i, t2))

3. Find the final list of k-frequent items (k-tuples) for k=3 and 4. Experiment a bit and describe the best value for the support in each case. *Warning*: You can use any of the three algorithms, but be careful, because the algorithm can take too long if you don't chose it properly (well, basically don't use the naïve approach ;)).

In [ ]:
#PCY for 3-tuples: k = 3
N=20
s = N
t = time.time()

def PCY3(s):
    t = time.time()
    # find frequent 1-tuples (individual items)
    C1 = {}
    for key in readdata(k=1, report=False):
        if key not in C1:
            C1[key] = 1
        else:
            C1[key] += 1    
            
    L1 = {}
    for key, count in C1.items():
        if count >= s:
            L1[key] = count
            
# find frequent 2-tuples
    C2 = {}
    for key in readdata(k=2, report=False):
     # record frequent tuples
        if key not in C2:
            C2[key] = 1
        else:
            C2[key] += 1

    # filter stage
    L2 = {}
    for key, count in C2.items():
        if count >= s:
            L2[key] = count

    C3_items = set([a.union(b) for a in L2.keys() for b in L2.keys() ]) # List comprehensions in python

    # hash table
    max_hash1 = 10 * 1000000
    H1 = np.zeros((max_hash1, ), dtype=int)

    for key in readdata(k=3, report=False):
        hash_cell_1 = hash(key) % max_hash1
        H1[hash_cell_1] += 1
    # find frequent 3-tuples
    C3 = {}

    for key in readdata(k=3):
        # hash-based filtering stage from PCY
        hash_cell_1 = hash(key) % max_hash1
        if H1[hash_cell_1] < s:
            continue
    
        # filter out non-frequent tuples
        if key not in C3_items:
            continue

        # record frequent tuples
        if key not in C3:
            C3[key] = 1
        else:
            C3[key] += 1

    # filter stage
    L3 = {}
    for key, count in C3.items():
        if count >= s:
            L3[key] = count
    t3 = round(time.time() - t,3)
    print('PCY3:      {} items with >{} occurances. Calculated in {} sec'.format(len(L3), s, t3))

In [ ]:
s=10
PCY3(s)

In [ ]:
#PCY for 4-tuples: k = 4
#N=20
#s = N
t = time.time()

def PCY4(s):
    t = time.time()
    # find frequent 1-tuples (individual items)
    C1 = {}
    for key in readdata(k=1, report=False):
        if key not in C1:
            C1[key] = 1
        else:
            C1[key] += 1    
            
    L1 = {}
    for key, count in C1.items():
        if count >= s:
            L1[key] = count
            
# find frequent 2-tuples
    C2 = {}
    for key in readdata(k=2, report=False):
     # record frequent tuples
        if key not in C2:
            C2[key] = 1
        else:
            C2[key] += 1

    # filter stage
    L2 = {}
    for key, count in C2.items():
        if count >= s:
            L2[key] = count

# find frequent 3-tuples
    C3 = {}
    for key in readdata(k=3, report=False):
     # record frequent tuples
        if key not in C3:
            C3[key] = 1
        else:
            C3[key] += 1

    # filter stage
    L3 = {}
    for key, count in C3.items():
        if count >= s:
            L3[key] = count
    
    C4_items = set([a.union(b) for a in L3.keys() for b in L3.keys() ]) # List comprehensions in python

    # hash table
    max_hash1 = 10 * 1000000
    H1 = np.zeros((max_hash1, ), dtype=int)

    for key in readdata(k=4, report=False):
        hash_cell_1 = hash(key) % max_hash1
        H1[hash_cell_1] += 1
    # find frequent 3-tuples
    C4 = {}

    for key in readdata(k=4):
        # hash-based filtering stage from PCY
        hash_cell_1 = hash(key) % max_hash1
        if H1[hash_cell_1] < s:
            continue
    
        # filter out non-frequent tuples
        if key not in C4_items:
            continue

        # record frequent tuples
        if key not in C4:
            C4[key] = 1
        else:
            C4[key] += 1

    # filter stage
    L4 = {}
    for key, count in C4.items():
        if count >= s:
            L4[key] = count
    t4 = round(time.time() - t,3)
    print('PCY4:      {} items with >{} occurances. Calculated in {} sec'.format(len(L4), s, t4))


In [ ]:
s=10
PCY4(s)

4. Using one of the results of the previous items, for one k (k=2 or 3) find the possible clusters using the 1-NN criteria. Comment your results.

In [21]:
s = 10
i = 2
k=2
start = time.time()
print("Calculating using pcy_multihash with s=%d, k=%d and %d hashtables"%(s,k,i))
res = C2
print("Found: %d abstracts in %.2fs"%(len(res),time.time() - start))

authorset = set()
for authors in res.keys():
    authorset = authorset.union(authors)
print("Found: %d authors"%len(authorset))

#create square bool grid to find matches
grid = np.zeros((len(authorset),len(authorset)), bool)
authorsmap = dict(enumerate(authorset))
authorsinvertedmap = {v: k for k, v in enumerate(authorset)}
for authors in res.keys():
    for authorpair in itertools.combinations(authors, 2):
        grid[authorsinvertedmap[authorpair[0]]][authorsinvertedmap[authorpair[1]]] = True

clusters = set()
for row in grid:
    cluster = set()
    for x,match in enumerate(row):
        if(match):
            cluster.add(authorsmap[x])
    if len(cluster) > 1:
        clusters.add(frozenset(cluster))

print("Found: %d clusters"%len(clusters))

Calculating using pcy_multihash with s=10, k=2 and 2 hashtables
Found: 263 abstracts in 0.00s
Found: 355 authors
Found: 42 clusters


In [25]:
clusters

{frozenset({'Anastasopoulos Antonios', 'Nakamura Satoshi', 'Sakti Sakriani'}),
 frozenset({'Niehues Jan', 'Waibel Alex'}),
 frozenset({"Blain Fr{\\'e}d{\\'e}ric", 'Scarton Carolina'}),
 frozenset({'Idiart Marco', 'Ramisch Carlos'}),
 frozenset({'Huck Matthias', 'Peitz Stephan'}),
 frozenset({'Nakamura Satoshi', 'Neubig Graham', 'Sakti Sakriani'}),
 frozenset({'Cho Eunah', 'Niehues Jan', 'Vogel Stephan'}),
 frozenset({'Li Mu', 'Zhou Ming'}),
 frozenset({'Bouillon Pierrette', 'Hockey Beth Ann'}),
 frozenset({'Li Peng', 'Zhang Jinchao'}),
 frozenset({'Liu Kang', 'Zhao Jun'}),
 frozenset({'Ney Hermann', 'Peitz Stephan'}),
 frozenset({'Wei Zhongyu', 'Zhang Qi'}),
 frozenset({'Glava{\\v{s}} Goran', 'Korhonen Anna', 'Reichart Roi'}),
 frozenset({'Allauzen Alexandre', 'Wisniewski Guillaume'}),
 frozenset({'Cettolo Mauro', 'Federico Marcello'}),
 frozenset({'Duan Nan', 'Zhou Ming'}),
 frozenset({'Panchenko Alexander', 'Riedl Martin', 'Yimam Seid Muhie'}),
 frozenset({'Bojar Ond{\\v{r}}ej',
    

## 5.	Conclusions

•	Summarize your findings here in 5...10 lines of text.

Here is the summary of our Frequent Item Sets finding steps performed:

1) Extract data from online source was our very first step. We extracted the given source data from .bib file and it needed few steps and cleaning to peform to get the required text file.

2) we loaded the file and then data mining was very critical and important task: As based on how we did the data mining. the ouput of data mining is further used for calculations. And hence this step is very much important for further modelling of the data.

3) We created frozensets of only authors & editors from abstract

4) For frequent item sets calculation, first of all we calcualated 1-tuple sets (k=1) which is used as very first input for further frequent item sets calculcations.

5) Then we used 3 different Frequent Item Sets methods namely (i) Naive (ii) A-Priori (iii) PCY. We performed Frequent Item Sets for 2-tuple (k=2) and for few support values (s) 10, 20, 100. And verified the results and peformances.
Observation: For smaller support values (10, 20 etc.), The Naive method is much faster as compared to A-Priori & PCY. However, as the higher support value are used (s = 50 to 100), the Naive method starts taking more processing time and A-Priori & PCY processing time gets reduced.
Hence, we can conclude, For the given data & for 2-tuple (k=2): 
-smaller the support values(s=10) faster processing time for Naive and slower processing time for A-Priori & PCY.
-larger the support values (s=50 to 100), slower the processing time for Naive and faster the processing time for A-Priori & PCY respetively.

6) We also calculated 3-tuple and 4-tuple (k=3 & k=4 respectively) frequet item sets using PCY method. And we see the results are good and peformace is fast enough.

7) Finally, clustering is done for 2-tuple frozen sets (k=2) using grid serach calculations. The clusters look good.

Overall, we conclude that the results of each step are satisfactory and good.

In [17]:
! git add Frequent_pairs.ipynb
! git commit -m "final comments"
! git push 

The file will have its original line endings in your working directory


[main 26d0de7] some progress
 1 file changed, 268 insertions(+), 465 deletions(-)


To https://github.com/AlexTouvras/FindingSimilarItems
   57a49d5..26d0de7  main -> main
